# Daily Résumé La Ferme de la Cour Bookings

In [11]:
# !pip install pandas openpyxl
# !pip install schedule
# !pip install gspread oauth2client

In [16]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def get_bookings_from_gsheet(credentials_file, spreadsheet_name):
    """Get booking data from Google Sheet"""
    # Define the scope
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    
    # Authenticate
    credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
    client = gspread.authorize(credentials)
    
    # Open the spreadsheet
    sheet = client.open(spreadsheet_name).worksheet("Sheet1")  # Adjust sheet name if needed
    
    # Get all data
    data = sheet.get_all_records()
    
    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    
    return df

In [17]:
# Uncomment and replace with your actual values when ready to use
df = get_bookings_from_gsheet("credentials.json", "lafermedelacour_bookings")
df.head()  # Display the first few rows to verify it works

ValueError: ('Unexpected credentials type', None, 'Expected', 'service_account')

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import locale

# Set locale to French for date formatting
try:
    locale.setlocale(locale.LC_TIME, 'fr_FR')  # Windows
    print("set locale woking successfully!")
except:
    try: 
        locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Linux/Mac
    except:
        pass  # If French locale is not available, default will be used

set locale woking successfully!


In [3]:
def format_date_french(date_str):
    """Format a date string to French format"""
    if isinstance(date_str, str):
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
    else:
        date_obj = date_str
    
    day_name = date_obj.strftime('%A').capitalize()
    day_num = date_obj.strftime('%d')
    month_name = date_obj.strftime('%B').capitalize()
    year = date_obj.strftime('%Y')
    
    return f"{day_name} {day_num} {month_name} {year}"

def load_and_preprocess_data(file_path):
    """Load and preprocess the booking data"""
    # Check file extension
    if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        # Use read_excel for Excel files
        df = pd.read_excel(file_path)
    else:
        # For CSV or TSV files, specify encoding to handle special characters
        df = pd.read_csv(file_path, delimiter='\t', encoding='utf-8', errors='replace')
    
    # Convert date columns to datetime
    date_columns = ['booking_date', 'arrival_date', 'departure_date', 
                   'cancellation_date', 'modification_date']
    
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')
    
    # Filter out cancelled bookings
    if 'status' in df.columns:
        df = df[df['status'] != 'Cancelled']
    
    return df

def get_bookings_for_date(df, target_date):
    """Get all bookings relevant for a specific date"""
    # Find bookings where the target date falls between arrival and departure (inclusive of arrival, exclusive of departure)
    mask = ((df['arrival_date'] <= target_date) & (df['departure_date'] > target_date))
    return df[mask]

def get_arrivals_for_date(df, target_date):
    """Get bookings arriving on the target date"""
    return df[df['arrival_date'] == target_date]

def get_departures_for_date(df, target_date):
    """Get bookings departing on the target date"""
    return df[df['departure_date'] == target_date]

def get_turnovers_for_date(df, target_date):
    """Get rooms with both arrival and departure on the same day"""
    departures = get_departures_for_date(df, target_date)
    arrivals = get_arrivals_for_date(df, target_date)
    
    turnover_rooms = []
    
    # Check each room column
    room_columns = [col for col in df.columns if col.startswith('room')]
    
    for room_col in room_columns:
        # Get departing rooms
        departing_rooms = departures[departures[room_col].notna()][room_col].unique()
        
        # Get arriving rooms
        arriving_rooms = arrivals[arrivals[room_col].notna()][room_col].unique()
        
        # Find rooms that have both arrival and departure
        for room in departing_rooms:
            if room in arriving_rooms and room not in turnover_rooms and room:
                turnover_rooms.append(room)
    
    # Get the relevant bookings for turnover rooms
    turnover_departures = departures[departures.apply(
        lambda row: any(room in turnover_rooms for room in row[room_columns] if pd.notna(room)), 
        axis=1
    )]
    
    turnover_arrivals = arrivals[arrivals.apply(
        lambda row: any(room in turnover_rooms for room in row[room_columns] if pd.notna(room)), 
        axis=1
    )]
    
    return turnover_rooms, turnover_departures, turnover_arrivals

def get_staying_bookings(df, target_date, arrivals, departures):
    """Get bookings that are staying (not arriving or departing)"""
    all_bookings = get_bookings_for_date(df, target_date)
    
    # Exclude arrivals and departures
    staying = all_bookings[~all_bookings['reference'].isin(arrivals['reference'].tolist() + departures['reference'].tolist())]
    
    return staying

def get_country_flag_code(nationality):
    """Return a country code for the given country"""
    country_to_code = {
        'Belgium': 'be',
        'France': 'fr',
        'Germany': 'de',
        'Netherlands': 'nl',
        'Luxembourg': 'lu',
        'United Kingdom': 'gb',
        'Italy': 'it',
        'Spain': 'es',
        'Portugal': 'pt',
        'Switzerland': 'ch',
        'Hungary': 'hu',
        # Add more as needed
    }
    
    return country_to_code.get(nationality, '')

def generate_room_card_html(booking, room_name, status, is_repeat=False, previous_notes=""):
    """Generate HTML for a room card"""
    
    # Determine badge and header class based on status
    if status == "arrival":
        badge_class = "badge-arrival"
        header_class = "arrival"
        badge_text = "Arrivée"
    elif status == "departure":
        badge_class = "badge-departure"
        header_class = "departure"
        badge_text = "Départ"
    elif status == "staying":
        badge_class = "badge-staying"
        header_class = "staying"
        badge_text = "Séjour"
    elif status == "turnover_departure":
        badge_class = "badge-departure"
        header_class = "departure"
        badge_text = "Départ"
    elif status == "turnover_arrival":
        badge_class = "badge-arrival"
        header_class = "arrival"
        badge_text = "Arrivée"
    else:
        badge_class = ""
        header_class = ""
        badge_text = ""
    
    # Format dates
    arrival_date = booking['arrival_date'].strftime('%d/%m/%Y') if pd.notna(booking['arrival_date']) else ""
    departure_date = booking['departure_date'].strftime('%d/%m/%Y') if pd.notna(booking['departure_date']) else ""
    
    # Get guest name
    guest_name = booking['guest_name'] if pd.notna(booking['guest_name']) else ""
    
    # Get nationality and flag code
    nationality = booking['nationality'] if pd.notna(booking['nationality']) else ""
    flag_code = get_country_flag_code(nationality)
    # Create flag display with CSS classes instead of emoji
    nationality_display = f'<span class="flag-icon flag-icon-{flag_code}"></span> {nationality}' if flag_code else nationality
    
    # Get contact info
    phone = booking['phone'] if pd.notna(booking['phone']) else ""
    email = booking['email'] if pd.notna(booking['email']) else ""
    
    # Get booking details
    amount = booking['amount'] if pd.notna(booking['amount']) else ""
    reference = booking['reference'] if pd.notna(booking['reference']) else ""
    source = booking['booking_source'] if pd.notna(booking['booking_source']) else ""
    
    # Get nights
    nights = booking['nights'] if pd.notna(booking['nights']) else ""
    
    # Get notes
    notes = booking['notes'] if pd.notna(booking['notes']) else ""
    
    # Get visit count
    visit_count = ""
    if pd.notna(booking.get('visit_count')) and booking['visit_count'] > 0:
        count = int(booking['visit_count'])
        if count == 1:
            visit_count = f'<div class="visit-count first-visit">Première visite</div>'
        else:
            visit_count = f'<div class="visit-count">{count}ème visite</div>'
    
    # Action needed text
    action_needed = ""
    if status == "arrival":
        action_needed = f'<div class="action-needed">Préparer la chambre "{room_name}"</div>'
    elif status == "departure":
        action_needed = '<div class="action-needed">C\'est leur dernier jour chez nous</div>'
    
    # Special styling for repeat guests
    repeat_guest_class = "repeat-guest" if is_repeat else ""
    repeat_guest_text = ""
    if is_repeat:
        repeat_guest_text = f'<div class="repeat-guest">Client fidèle! Visite précédente: {previous_notes}</div>'
    
    # Generate the HTML
    html = f'''
    <div class="room-card">
        <div class="room-header {header_class}">
            <span>Chambre: {room_name}</span>
            <span class="badge {badge_class}">{badge_text}</span>
        </div>
        <div class="room-content">
            <div class="guest-info">
                <div class="info-group">
                    <div><span class="info-label">Nom:</span> <span class="info-value {repeat_guest_class}">{guest_name}</span></div>
                    <div><span class="info-label">Nationalité:</span> <span class="info-value">{nationality_display}</span></div>
                    <div><span class="info-label">Téléphone:</span> <span class="info-value">{phone}</span></div>
                    <div><span class="info-label">Email:</span> <span class="info-value">{email}</span></div>
                </div>
                <div class="info-group">
                    <div><span class="info-label">Arrivée:</span> <span class="info-value">{arrival_date}</span></div>
                    <div><span class="info-label">Départ:</span> <span class="info-value">{departure_date}</span></div>
                    <div><span class="info-label">Nuits:</span> <span class="info-value">{nights}</span></div>
                </div>
                <div class="info-group">
                    <div><span class="info-label">Montant:</span> <span class="info-value">{amount} €</span></div>
                    <div><span class="info-label">Référence:</span> <span class="info-value">{reference}</span></div>
                    <div><span class="info-label">Source:</span> <span class="info-value">{source}</span></div>
                </div>
            </div>
            {action_needed}
            {repeat_guest_text}
            {visit_count}
            <div class="notes">
                Notes: {notes}
            </div>
        </div>
    </div>
    '''
    
    return html

def generate_turnover_card_html(departure_booking, arrival_booking, room_name):
    """Generate HTML for a turnover room card (departure and arrival on same day)"""
    
    # Format dates for departure booking
    dep_arrival_date = departure_booking['arrival_date'].strftime('%d/%m/%Y') if pd.notna(departure_booking['arrival_date']) else ""
    dep_departure_date = departure_booking['departure_date'].strftime('%d/%m/%Y') if pd.notna(departure_booking['departure_date']) else ""
    
    # Format dates for arrival booking
    arr_arrival_date = arrival_booking['arrival_date'].strftime('%d/%m/%Y') if pd.notna(arrival_booking['arrival_date']) else ""
    arr_departure_date = arrival_booking['departure_date'].strftime('%d/%m/%Y') if pd.notna(arrival_booking['departure_date']) else ""
    
    # Get guest names
    dep_guest_name = departure_booking['guest_name'] if pd.notna(departure_booking['guest_name']) else ""
    arr_guest_name = arrival_booking['guest_name'] if pd.notna(arrival_booking['guest_name']) else ""
    
    # Get nationalities and flags - using CSS flags now
    dep_nationality = departure_booking['nationality'] if pd.notna(departure_booking['nationality']) else ""
    dep_flag_code = get_country_flag_code(dep_nationality)
    dep_nationality_display = f'<span class="flag-icon flag-icon-{dep_flag_code}"></span> {dep_nationality}' if dep_flag_code else dep_nationality
    
    arr_nationality = arrival_booking['nationality'] if pd.notna(arrival_booking['nationality']) else ""
    arr_flag_code = get_country_flag_code(arr_nationality)
    arr_nationality_display = f'<span class="flag-icon flag-icon-{arr_flag_code}"></span> {arr_nationality}' if arr_flag_code else arr_nationality
    
    # Get contact info
    dep_phone = departure_booking['phone'] if pd.notna(departure_booking['phone']) else ""
    arr_phone = arrival_booking['phone'] if pd.notna(arrival_booking['phone']) else ""
    
    dep_email = departure_booking['email'] if pd.notna(departure_booking['email']) else ""
    arr_email = arrival_booking['email'] if pd.notna(arrival_booking['email']) else ""
    
    # Get notes
    dep_notes = departure_booking['notes'] if pd.notna(departure_booking['notes']) else ""
    arr_notes = arrival_booking['notes'] if pd.notna(arrival_booking['notes']) else ""
    
    # Get visit counts
    dep_visit_count = ""
    if pd.notna(departure_booking.get('visit_count')) and departure_booking['visit_count'] > 0:
        count = int(departure_booking['visit_count'])
        if count == 1:
            dep_visit_count = f'<div class="visit-count first-visit">Première visite</div>'
        else:
            dep_visit_count = f'<div class="visit-count">{count}ème visite</div>'
    
    arr_visit_count = ""
    if pd.notna(arrival_booking.get('visit_count')) and arrival_booking['visit_count'] > 0:
        count = int(arrival_booking['visit_count'])
        if count == 1:
            arr_visit_count = f'<div class="visit-count first-visit">Première visite</div>'
        else:
            arr_visit_count = f'<div class="visit-count">{count}ème visite</div>'
    
    # Generate the HTML
    html = f'''
    <div class="room-card">
        <div class="room-header turnover">
            <span>Chambre: {room_name}</span>
            <span class="badge badge-turnover">Changement</span>
        </div>
        <div class="room-content">
            <h4>Départ:</h4>
            <div class="guest-info">
                <div class="info-group">
                    <div><span class="info-label">Nom:</span> <span class="info-value">{dep_guest_name}</span></div>
                    <div><span class="info-label">Nationalité:</span> <span class="info-value">{dep_nationality_display}</span></div>
                    <div><span class="info-label">Téléphone:</span> <span class="info-value">{dep_phone}</span></div>
                    <div><span class="info-label">Email:</span> <span class="info-value">{dep_email}</span></div>
                </div>
                <div class="info-group">
                    <div><span class="info-label">Arrivée:</span> <span class="info-value">{dep_arrival_date}</span></div>
                    <div><span class="info-label">Départ:</span> <span class="info-value">{dep_departure_date}</span></div>
                </div>
            </div>
            
            {dep_visit_count}
            <div class="notes">
                Notes: {dep_notes}
            </div>
            
            <h4>Arrivée:</h4>
            <div class="guest-info">
                <div class="info-group">
                    <div><span class="info-label">Nom:</span> <span class="info-value">{arr_guest_name}</span></div>
                    <div><span class="info-label">Nationalité:</span> <span class="info-value">{arr_nationality_display}</span></div>
                    <div><span class="info-label">Téléphone:</span> <span class="info-value">{arr_phone}</span></div>
                    <div><span class="info-label">Email:</span> <span class="info-value">{arr_email}</span></div>
                </div>
                <div class="info-group">
                    <div><span class="info-label">Arrivée:</span> <span class="info-value">{arr_arrival_date}</span></div>
                    <div><span class="info-label">Départ:</span> <span class="info-value">{arr_departure_date}</span></div>
                </div>
            </div>
            
            {arr_visit_count}
            <div class="notes">
                Notes: {arr_notes}
            </div>
            
            <div class="action-needed">
                Nettoyer et préparer la chambre pour les nouveaux hôtes
            </div>
        </div>
    </div>
    '''
    
    return html

def generate_html_for_date(df, target_date, logo_path=None, user_name="Juliette Maes"):
    """Generate the complete HTML for a specific date"""
    
    # Get the formatted date
    formatted_date = format_date_french(target_date)
    generation_date = datetime.now().strftime('%d/%m/%Y %H:%M')
    
    # Get all bookings for the date
    arrivals = get_arrivals_for_date(df, target_date)
    departures = get_departures_for_date(df, target_date)
    
    # Get turnover rooms (rooms with both arrival and departure on the same day)
    turnover_rooms, turnover_departures, turnover_arrivals = get_turnovers_for_date(df, target_date)
    
    # Filter out turnovers from regular arrivals and departures
    if not turnover_departures.empty and not turnover_arrivals.empty:
        arrivals = arrivals[~arrivals['reference'].isin(turnover_arrivals['reference'])]
        departures = departures[~departures['reference'].isin(turnover_departures['reference'])]
    
    # Get staying bookings (not arriving or departing)
    staying = get_staying_bookings(df, target_date, arrivals, departures)
    
    # Count bookings
    num_arrivals = len(arrivals) + len(turnover_arrivals)
    num_departures = len(departures) + len(turnover_departures)
    num_staying = len(staying)
    
    # Prepare logo HTML
    logo_html = ""
    if logo_path:
        if os.path.exists(logo_path):
            # If using base64 encoding (makes HTML self-contained)
            import base64
            with open(logo_path, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                ext = os.path.splitext(logo_path)[1][1:]  # Get extension without dot
                logo_html = f'<div class="logo"><img src="data:image/{ext};base64,{encoded_string}" alt="La Ferme de la Cour Logo"></div>'
        else:
            # If referencing the logo file directly (requires logo to be in same folder as HTML)
            logo_filename = os.path.basename(logo_path)
            logo_html = f'<div class="logo"><img src="{logo_filename}" alt="La Ferme de la Cour Logo"></div>'
    
    # Start generating the HTML
    html_template = """<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>La Ferme de la Cour - Résumé Journalier</title>
    <!-- Add flag-icon-css library -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/flag-icon-css/3.5.0/css/flag-icon.min.css">
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
            color: #333;
        }
        .container {
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
        }
        .header {
            background-color: #698B69;
            color: white;
            padding: 20px;
            text-align: center;
            border-radius: 10px 10px 0 0;
            margin-bottom: 20px;
        }
        .logo {
            margin-bottom: 15px;
        }
        .logo img {
            max-height: 100px;
            max-width: 100%;
        }
        .date-display {
            font-size: 28px;
            margin-bottom: 10px;
        }
        .summary {
            background-color: #f0f8ff;
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 20px;
            text-align: center;
            font-size: 18px;
        }
        .no-booking {
            background-color: #e8f5e9;
            color: #2e7d32;
            padding: 30px;
            border-radius: 10px;
            text-align: center;
            font-size: 24px;
        }
        .booking-section {
            margin-bottom: 25px;
        }
        .section-title {
            background-color: #698B69;
            color: white;
            padding: 10px 15px;
            border-radius: 5px;
            font-size: 18px;
        }
        .room-card {
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            margin-bottom: 15px;
            overflow: hidden;
        }
        .room-header {
            padding: 12px 15px;
            font-weight: bold;
            font-size: 18px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }
        .arrival {
            background-color: #e3f2fd;
            border-left: 5px solid #2196f3;
        }
        .departure {
            background-color: #ffebee;
            border-left: 5px solid #f44336;
        }
        .staying {
            background-color: #e8f5e9;
            border-left: 5px solid #4caf50;
        }
        .turnover {
            background-color: #fff3e0;
            border-left: 5px solid #ff9800;
        }
        .room-content {
            padding: 15px;
        }
        .guest-info {
            display: flex;
            flex-wrap: wrap;
            margin-bottom: 10px;
        }
        .info-group {
            flex: 1;
            min-width: 200px;
            margin-bottom: 10px;
        }
        .info-label {
            font-weight: bold;
            color: #555;
            margin-right: 5px;
        }
        .info-value {
            color: #333;
        }
        .action-needed {
            background-color: #fff8e1;
            padding: 10px 15px;
            margin-top: 10px;
            border-radius: 5px;
            font-weight: bold;
            border-left: 5px solid #ffc107;
        }
        .notes {
            background-color: #f9f9f9;
            padding: 10px 15px;
            margin-top: 10px;
            border-radius: 5px;
            font-style: italic;
        }
        .repeat-guest {
            color: #7b1fa2;
            font-weight: bold;
        }
        .badge {
            display: inline-block;
            padding: 3px 8px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: bold;
            color: white;
        }
        .badge-arrival {
            background-color: #2196f3;
        }
        .badge-departure {
            background-color: #f44336;
        }
        .badge-staying {
            background-color: #4caf50;
        }
        .badge-turnover {
            background-color: #ff9800;
        }
        .visit-count {
            background-color: #e8eaf6;
            padding: 8px 12px;
            margin-top: 10px;
            border-radius: 5px;
            font-weight: bold;
            border-left: 5px solid #3f51b5;
        }
        .first-visit {
            background-color: #e0f7fa;
            border-left: 5px solid #00bcd4;
        }
        .footer {
            text-align: center;
            padding: 20px;
            font-size: 14px;
            color: #666;
        }
        /* Style for flag icons */
        .flag-icon {
            margin-right: 5px;
        }
        @media (max-width: 600px) {
            .info-group {
                min-width: 100%;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            {LOGO_HTML}
            <div class="date-display">{TODAY_DATE_LONG}</div>
            <div>La Ferme de la Cour</div>
        </div>
    """
    
    # Replace placeholders
    html_template = html_template.replace("{TODAY_DATE_LONG}", formatted_date)
    html_template = html_template.replace("{LOGO_HTML}", logo_html)
    
    # If no bookings for the day
    if num_arrivals == 0 and num_departures == 0 and num_staying == 0:
        html_template += """
        <div class="no-booking">
            <p>Aujourd'hui c'est repos! 😊</p>
            <p>Aucune réservation pour aujourd'hui.</p>
        </div>
        """
    else:
        # Add summary
        html_template += f"""
        <div class="summary">
            <p>Aujourd'hui: {num_arrivals} arrivée(s), {num_departures} départ(s), {num_staying} séjour(s) en cours</p>
        </div>
        """
        
        # Add arrival section if there are arrivals
        if num_arrivals > 0:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🛎️ Arrivées Aujourd'hui</h2>
            """
            
            # Add room cards for arrivals
            for _, booking in arrivals.iterrows():
                # Find which room this booking is for
                room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                for room_col in room_columns:
                    if pd.notna(booking[room_col]):
                        room_name = booking[room_col]
                        is_repeat = booking['repeat_guest'] if 'repeat_guest' in booking and pd.notna(booking['repeat_guest']) else False
                        previous_notes = booking['notes'] if 'notes' in booking and pd.notna(booking['notes']) and is_repeat else ""
                        
                        html_template += generate_room_card_html(booking, room_name, "arrival", is_repeat, previous_notes)
            
            html_template += """
            </div>
            """
        
        # Add departure section if there are departures
        if num_departures > 0:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🚪 Départs Aujourd'hui</h2>
            """
            
            # Add room cards for departures
            for _, booking in departures.iterrows():
                # Find which room this booking is for
                room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                for room_col in room_columns:
                    if pd.notna(booking[room_col]):
                        room_name = booking[room_col]
                        is_repeat = booking['repeat_guest'] if 'repeat_guest' in booking and pd.notna(booking['repeat_guest']) else False
                        previous_notes = booking['notes'] if 'notes' in booking and pd.notna(booking['notes']) and is_repeat else ""
                        
                        html_template += generate_room_card_html(booking, room_name, "departure", is_repeat, previous_notes)
            
            html_template += """
            </div>
            """
        
        # Add staying section if there are staying bookings
        if num_staying > 0:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🏠 Séjours en Cours</h2>
            """
            
            # Add room cards for staying bookings
            for _, booking in staying.iterrows():
                # Find which room this booking is for
                room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                for room_col in room_columns:
                    if pd.notna(booking[room_col]):
                        room_name = booking[room_col]
                        is_repeat = booking['repeat_guest'] if 'repeat_guest' in booking and pd.notna(booking['repeat_guest']) else False
                        previous_notes = booking['notes'] if 'notes' in booking and pd.notna(booking['notes']) and is_repeat else ""
                        
                        html_template += generate_room_card_html(booking, room_name, "staying", is_repeat, previous_notes)
            
            html_template += """
            </div>
            """
        
        # Add turnover section if there are turnovers
        if turnover_rooms:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🔄 Chambres à Préparer (Départ & Arrivée)</h2>
            """
            
            # For each turnover room, find the corresponding departure and arrival bookings
            for room in turnover_rooms:
                # Find departure booking for this room
                dep_booking = None
                for _, booking in turnover_departures.iterrows():
                    room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                    for room_col in room_columns:
                        if pd.notna(booking[room_col]) and booking[room_col] == room:
                            dep_booking = booking
                            break
                    if dep_booking is not None:
                        break
                
                # Find arrival booking for this room
                arr_booking = None
                for _, booking in turnover_arrivals.iterrows():
                    room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                    for room_col in room_columns:
                        if pd.notna(booking[room_col]) and booking[room_col] == room:
                            arr_booking = booking
                            break
                    if arr_booking is not None:
                        break
                
                # Generate turnover card HTML
                if dep_booking is not None and arr_booking is not None:
                    html_template += generate_turnover_card_html(dep_booking, arr_booking, room)
            
            html_template += """
            </div>
            """
    
    # Update footer with custom text
    html_template += f"""
        <div class="footer">
            <p>La Ferme de la Cour - Résumé journalier généré le {generation_date} par {user_name}</p>
        </div>
    </div>
</body>
</html>
    """
    
    return html_template

def generate_daily_html_files(df, start_date, end_date, output_dir="daily_html", logo_path=None, user_name="Juliette Maes"):
    """Generate HTML files for a range of dates"""
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate HTML for each day in the range
    current_date = start_date
    while current_date <= end_date:
        html_content = generate_html_for_date(df, current_date, logo_path, user_name)
        
        # Save HTML to file
        file_name = f"{current_date.strftime('%Y-%m-%d')}.html"
        file_path = os.path.join(output_dir, file_name)
        
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"Generated {file_path}")
        
        # Move to next day
        current_date += timedelta(days=1)

## Weekly Summaries

In [4]:
import pandas as pd
from datetime import datetime, timedelta
import os
import calendar

def format_date_range_french(start_date, end_date):
    """Format a date range in French format"""
    start_day_name = start_date.strftime('%A').capitalize()
    start_day_num = start_date.strftime('%d')
    start_month_name = start_date.strftime('%B').capitalize()
    
    end_day_name = end_date.strftime('%A').capitalize()
    end_day_num = end_date.strftime('%d')
    end_month_name = end_date.strftime('%B').capitalize()
    year = start_date.strftime('%Y')
    
    # If same month
    if start_month_name == end_month_name:
        return f"Semaine du {start_day_name} {start_day_num} au {end_day_name} {end_day_num} {end_month_name} {year}"
    else:
        return f"Semaine du {start_day_name} {start_day_num} {start_month_name} au {end_day_name} {end_day_num} {end_month_name} {year}"

def get_week_bookings(df, start_date, end_date):
    """Get all bookings relevant for a specific week"""
    # Bookings that overlap with the week
    mask = ~(
        (df['arrival_date'] > end_date) | 
        (df['departure_date'] <= start_date)
    )
    return df[mask]

def get_week_arrivals(df, start_date, end_date):
    """Get bookings arriving during the week"""
    return df[(df['arrival_date'] >= start_date) & (df['arrival_date'] <= end_date)]

def get_week_departures(df, start_date, end_date):
    """Get bookings departing during the week"""
    return df[(df['departure_date'] >= start_date) & (df['departure_date'] <= end_date)]

def get_repeat_guests(df, start_date, end_date):
    """Get repeat guests arriving during the week"""
    week_bookings = get_week_bookings(df, start_date, end_date)
    if 'repeat_guest' in week_bookings.columns:
        return week_bookings[week_bookings['repeat_guest'] == True]
    return pd.DataFrame()  # Return empty DataFrame if no repeat_guest column

def get_room_occupancy_stats(df, start_date, end_date):
    """Calculate room occupancy statistics for the week"""
    # Get all room columns
    room_columns = [col for col in df.columns if col.startswith('room')]
    
    # Initialize counters
    room_stats = {}
    total_rooms = len(room_columns)
    
    # Calculate occupancy for each day of the week
    current_date = start_date
    daily_occupancy = {}
    
    while current_date <= end_date:
        bookings_on_date = get_bookings_for_date(df, current_date)
        occupied_rooms = set()
        
        for _, booking in bookings_on_date.iterrows():
            for room_col in room_columns:
                if pd.notna(booking[room_col]):
                    room_name = booking[room_col]
                    occupied_rooms.add(room_name)
        
        # Calculate occupancy percentage
        day_occupancy = (len(occupied_rooms) / total_rooms) * 100 if total_rooms > 0 else 0
        day_name = current_date.strftime('%A').capitalize()
        daily_occupancy[day_name] = {
            'date': current_date.strftime('%d/%m/%Y'),
            'occupied': len(occupied_rooms),
            'total': total_rooms,
            'percentage': day_occupancy
        }
        
        # Move to next day
        current_date += timedelta(days=1)
    
    return daily_occupancy

def get_turnover_days(df, start_date, end_date):
    """Get days with turnovers (same room has departure and arrival)"""
    turnover_days = {}
    
    current_date = start_date
    while current_date <= end_date:
        turnover_rooms, turnover_departures, turnover_arrivals = get_turnovers_for_date(df, current_date)
        
        if turnover_rooms:
            day_name = current_date.strftime('%A').capitalize()
            turnover_days[day_name] = {
                'date': current_date.strftime('%d/%m/%Y'),
                'rooms': turnover_rooms,
                'count': len(turnover_rooms)
            }
        
        current_date += timedelta(days=1)
    
    return turnover_days

def calculate_revenue_for_week(df, start_date, end_date):
    """Calculate revenue stats for the week"""
    week_bookings = get_week_bookings(df, start_date, end_date)
    
    # Filter for bookings with arrival during the week
    week_arrivals = get_week_arrivals(df, start_date, end_date)
    
    total_revenue = 0
    if 'amount' in week_arrivals.columns:
        # Sum amounts for bookings arriving this week
        total_revenue = week_arrivals['amount'].sum() if not week_arrivals.empty else 0
    
    # Count bookings by source
    source_counts = {}
    if 'booking_source' in week_arrivals.columns:
        source_counts = week_arrivals['booking_source'].value_counts().to_dict()
    
    return {
        'total_revenue': total_revenue,
        'source_counts': source_counts
    }

def generate_weekly_summary_html(df, start_date, logo_path=None, user_name="Juliette Maes"):
    """Generate weekly summary HTML for the week starting at start_date"""
    # Calculate end date (Sunday)
    end_date = start_date + timedelta(days=6)
    
    # Format the date range
    date_range = format_date_range_french(start_date, end_date)
    generation_date = datetime.now().strftime('%d/%m/%Y %H:%M')
    
    # Get week's bookings
    week_bookings = get_week_bookings(df, start_date, end_date)
    week_arrivals = get_week_arrivals(df, start_date, end_date)
    week_departures = get_week_departures(df, start_date, end_date)
    repeat_guests = get_repeat_guests(df, start_date, end_date)
    
    # Get occupancy stats
    occupancy_stats = get_room_occupancy_stats(df, start_date, end_date)
    
    # Get turnover days
    turnover_days = get_turnover_days(df, start_date, end_date)
    
    # Get revenue stats
    revenue_stats = calculate_revenue_for_week(df, start_date, end_date)
    
    # Prepare logo HTML
    logo_html = ""
    if logo_path:
        if os.path.exists(logo_path):
            # If using base64 encoding (makes HTML self-contained)
            import base64
            with open(logo_path, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                ext = os.path.splitext(logo_path)[1][1:]  # Get extension without dot
                logo_html = f'<div class="logo"><img src="data:image/{ext};base64,{encoded_string}" alt="La Ferme de la Cour Logo"></div>'
        else:
            # If referencing the logo file directly (requires logo to be in same folder as HTML)
            logo_filename = os.path.basename(logo_path)
            logo_html = f'<div class="logo"><img src="{logo_filename}" alt="La Ferme de la Cour Logo"></div>'
    
    # Start generating the HTML
    html_template = """<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>La Ferme de la Cour - Résumé Hebdomadaire</title>
    <!-- Add flag-icon-css library -->
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/flag-icon-css/3.5.0/css/flag-icon.min.css">
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
            color: #333;
        }
        .container {
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
        }
        .header {
            background-color: #698B69;
            color: white;
            padding: 20px;
            text-align: center;
            border-radius: 10px 10px 0 0;
            margin-bottom: 20px;
        }
        .logo {
            margin-bottom: 15px;
        }
        .logo img {
            max-height: 100px;
            max-width: 100%;
        }
        .date-display {
            font-size: 28px;
            margin-bottom: 10px;
        }
        .summary {
            background-color: #f0f8ff;
            padding: 15px;
            border-radius: 10px;
            margin-bottom: 20px;
            text-align: center;
            font-size: 18px;
        }
        .no-booking {
            background-color: #e8f5e9;
            color: #2e7d32;
            padding: 30px;
            border-radius: 10px;
            text-align: center;
            font-size: 24px;
        }
        .booking-section {
            margin-bottom: 25px;
        }
        .section-title {
            background-color: #698B69;
            color: white;
            padding: 10px 15px;
            border-radius: 5px;
            font-size: 18px;
            margin-bottom: 15px;
        }
        .stat-grid {
            display: grid;
            grid-template-columns: repeat(auto-fill, minmax(250px, 1fr));
            gap: 15px;
            margin-bottom: 20px;
        }
        .stat-card {
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            padding: 15px;
            text-align: center;
        }
        .stat-value {
            font-size: 24px;
            font-weight: bold;
            margin: 10px 0;
            color: #2e7d32;
        }
        .occupancy-table {
            width: 100%;
            border-collapse: collapse;
            margin-bottom: 20px;
            background-color: white;
            border-radius: 10px;
            overflow: hidden;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .occupancy-table th, .occupancy-table td {
            padding: 12px 15px;
            text-align: center;
            border-bottom: 1px solid #ddd;
        }
        .occupancy-table th {
            background-color: #698B69;
            color: white;
        }
        .occupancy-table tr:nth-child(even) {
            background-color: #f9f9f9;
        }
        .occupancy-meter {
            width: 100%;
            height: 10px;
            background-color: #e0e0e0;
            border-radius: 5px;
            overflow: hidden;
            margin-top: 5px;
        }
        .occupancy-fill {
            height: 100%;
            background-color: #4caf50;
        }
        .guest-list {
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            padding: 15px;
            margin-bottom: 15px;
        }
        .guest-item {
            margin-bottom: 10px;
            padding-bottom: 10px;
            border-bottom: 1px solid #eee;
        }
        .guest-item:last-child {
            border-bottom: none;
            margin-bottom: 0;
            padding-bottom: 0;
        }
        .guest-name {
            font-weight: bold;
        }
        .guest-info {
            margin-top: 5px;
            display: flex;
            flex-wrap: wrap;
        }
        .guest-info span {
            margin-right: 15px;
            margin-bottom: 5px;
        }
        .repeat-guest-tag {
            background-color: #e8f5e9;
            color: #2e7d32;
            padding: 3px 8px;
            border-radius: 12px;
            font-size: 12px;
            font-weight: bold;
            margin-left: 5px;
        }
        .turnover-day {
            background-color: #fff3e0;
            border-left: 5px solid #ff9800;
            padding: 10px 15px;
            margin-bottom: 10px;
            border-radius: 5px;
        }
        .turnover-info {
            margin-top: 5px;
            font-style: italic;
        }
        .footer {
            text-align: center;
            padding: 20px;
            font-size: 14px;
            color: #666;
        }
        .pie-chart-container {
            display: flex;
            justify-content: space-around;
            flex-wrap: wrap;
            margin: 20px 0;
        }
        .pie-chart {
            width: 150px;
            height: 150px;
            position: relative;
        }
        .legend {
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            margin-top: 15px;
        }
        .legend-item {
            display: flex;
            align-items: center;
            margin-right: 15px;
            margin-bottom: 5px;
        }
        .legend-color {
            width: 12px;
            height: 12px;
            margin-right: 5px;
            border-radius: 2px;
        }
        @media (max-width: 600px) {
            .stat-grid {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            {LOGO_HTML}
            <div class="date-display">{WEEK_DATE_RANGE}</div>
            <div>La Ferme de la Cour</div>
        </div>
    """
    
    # Replace placeholders
    html_template = html_template.replace("{WEEK_DATE_RANGE}", date_range)
    html_template = html_template.replace("{LOGO_HTML}", logo_html)
    
    # If no bookings for the week
    if len(week_bookings) == 0:
        html_template += """
        <div class="no-booking">
            <p>Semaine tranquille! 😊</p>
            <p>Aucune réservation pour cette semaine.</p>
        </div>
        """
    else:
        # Add summary stats
        html_template += f"""
        <div class="summary">
            <p>Cette semaine: {len(week_arrivals)} arrivée(s), {len(week_departures)} départ(s), {len(week_bookings)} séjour(s) total</p>
        </div>
        
        <div class="booking-section">
            <h2 class="section-title">📊 Statistiques de la Semaine</h2>
            <div class="stat-grid">
                <div class="stat-card">
                    <div>Arrivées</div>
                    <div class="stat-value">{len(week_arrivals)}</div>
                </div>
                <div class="stat-card">
                    <div>Départs</div>
                    <div class="stat-value">{len(week_departures)}</div>
                </div>
                <div class="stat-card">
                    <div>Clients Fidèles</div>
                    <div class="stat-value">{len(repeat_guests)}</div>
                </div>
                <div class="stat-card">
                    <div>Revenu Estimé</div>
                    <div class="stat-value">{revenue_stats['total_revenue']} €</div>
                </div>
            </div>
        </div>
        """
        
        # Add occupancy table section
        html_template += """
        <div class="booking-section">
            <h2 class="section-title">🏠 Taux d'Occupation</h2>
            <table class="occupancy-table">
                <thead>
                    <tr>
                        <th>Jour</th>
                        <th>Date</th>
                        <th>Chambres Occupées</th>
                        <th>Taux</th>
                        <th>Occupation</th>
                    </tr>
                </thead>
                <tbody>
        """
        
        # Add rows for each day
        for day, stats in occupancy_stats.items():
            occupancy_percentage = stats['percentage']
            html_template += f"""
                <tr>
                    <td>{day}</td>
                    <td>{stats['date']}</td>
                    <td>{stats['occupied']}/{stats['total']}</td>
                    <td>{occupancy_percentage:.0f}%</td>
                    <td>
                        <div class="occupancy-meter">
                            <div class="occupancy-fill" style="width: {occupancy_percentage}%"></div>
                        </div>
                    </td>
                </tr>
            """
        
        html_template += """
                </tbody>
            </table>
        </div>
        """
        
        # Add booking source breakdown if we have data
        if revenue_stats['source_counts']:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">📱 Sources de Réservation</h2>
                <div class="guest-list">
            """
            
            # Add source counts
            for source, count in revenue_stats['source_counts'].items():
                html_template += f"""
                    <div class="guest-item">
                        <div class="guest-name">{source}</div>
                        <div class="guest-info">
                            <span>Nombre de réservations: {count}</span>
                        </div>
                    </div>
                """
            
            html_template += """
                </div>
            </div>
            """
        
        # Add arrival list section
        if not week_arrivals.empty:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🛎️ Arrivées Cette Semaine</h2>
                <div class="guest-list">
            """
            
            # Add arrival items
            for _, booking in week_arrivals.iterrows():
                # Get guest info
                guest_name = booking['guest_name'] if pd.notna(booking['guest_name']) else "Invité"
                arrival_date = booking['arrival_date'].strftime('%A %d/%m/%Y').capitalize() if pd.notna(booking['arrival_date']) else ""
                departure_date = booking['departure_date'].strftime('%d/%m/%Y') if pd.notna(booking['departure_date']) else ""
                nights = booking['nights'] if pd.notna(booking.get('nights')) else (booking['departure_date'] - booking['arrival_date']).days if pd.notna(booking['departure_date']) and pd.notna(booking['arrival_date']) else ""
                
                # Get room info
                room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                rooms = [booking[room_col] for room_col in room_columns if pd.notna(booking[room_col])]
                rooms_text = ", ".join(rooms) if rooms else "Non spécifié"
                
                # Check if repeat guest
                is_repeat = booking.get('repeat_guest', False) if pd.notna(booking.get('repeat_guest', False)) else False
                repeat_tag = '<span class="repeat-guest-tag">Client Fidèle</span>' if is_repeat else ''
                
                html_template += f"""
                    <div class="guest-item">
                        <div class="guest-name">{guest_name} {repeat_tag}</div>
                        <div class="guest-info">
                            <span>Arrivée: {arrival_date}</span>
                            <span>Départ: {departure_date}</span>
                            <span>Nuits: {nights}</span>
                            <span>Chambre(s): {rooms_text}</span>
                        </div>
                    </div>
                """
            
            html_template += """
                </div>
            </div>
            """
        
        # Add turnover days section
        if turnover_days:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🔄 Jours de Changement</h2>
            """
            
            # Add turnover day items
            for day, turnover in turnover_days.items():
                rooms_text = ", ".join(turnover['rooms']) if turnover['rooms'] else ""
                html_template += f"""
                <div class="turnover-day">
                    <div><strong>{day} {turnover['date']}</strong> - {turnover['count']} chambre(s) à préparer</div>
                    <div class="turnover-info">Chambres: {rooms_text}</div>
                </div>
                """
            
            html_template += """
            </div>
            """
        
        # Add repeat guests section
        if not repeat_guests.empty:
            html_template += """
            <div class="booking-section">
                <h2 class="section-title">🌟 Clients Fidèles Cette Semaine</h2>
                <div class="guest-list">
            """
            
            # Add repeat guest items
            for _, booking in repeat_guests.iterrows():
                # Get guest info
                guest_name = booking['guest_name'] if pd.notna(booking['guest_name']) else "Invité"
                arrival_date = booking['arrival_date'].strftime('%A %d/%m/%Y').capitalize() if pd.notna(booking['arrival_date']) else ""
                previous_notes = booking['notes'] if pd.notna(booking.get('notes')) else "Pas de notes précédentes"
                
                # Get room info
                room_columns = [col for col in booking.index if col.startswith('room') and pd.notna(booking[col])]
                rooms = [booking[room_col] for room_col in room_columns if pd.notna(booking[room_col])]
                rooms_text = ", ".join(rooms) if rooms else "Non spécifié"
                
                html_template += f"""
                    <div class="guest-item">
                        <div class="guest-name">{guest_name}</div>
                        <div class="guest-info">
                            <span>Arrivée: {arrival_date}</span>
                            <span>Chambre(s): {rooms_text}</span>
                        </div>
                        <div style="margin-top: 5px; font-style: italic;">
                            Notes précédentes: {previous_notes}
                        </div>
                    </div>
                """
            
            html_template += """
                </div>
            </div>
            """
    
    # Add footer
    html_template += f"""
        <div class="footer">
            <p>La Ferme de la Cour - Résumé hebdomadaire généré le {generation_date} par {user_name}</p>
        </div>
    </div>
</body>
</html>
    """
    
    return html_template

def generate_weekly_html_files(df, start_date, end_date, output_dir="weekly_html", logo_path=None, user_name="Juliette Maes"):
    """Generate weekly HTML files from start_date to end_date"""
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Find first Monday from start_date
    days_to_add = (7 - start_date.weekday()) % 7
    if days_to_add == 7:  # If already Monday
        first_monday = start_date
    else:
        first_monday = start_date + timedelta(days=days_to_add)
    
    # Generate weekly HTML files starting from each Monday
    current_monday = first_monday
    while current_monday <= end_date:
        html_content = generate_weekly_summary_html(df, current_monday, logo_path, user_name)
        
        # Save HTML to file
        file_name = f"weekly_{current_monday.strftime('%Y-%m-%d')}.html"
        file_path = os.path.join(output_dir, file_name)
        
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(html_content)
        
        print(f"Generated weekly summary: {file_path}")
        
        # Move to next Monday
        current_monday += timedelta(days=7)

In [5]:
def main():
    # Set the file path to your booking data
    try:
        file_path = "lafermedelacour_bookings.xlsx"
    except:
        file_path = r"C:\Users\jules\LaFermedelaCour\lafermedelacour_bookings.xlsx"
    
    # Path to your logo (set to None if you don't have one)
    logo_path = "logo.png"  # Change to your logo path or set to None
    
    # Your name for the footer
    user_name = "Juliette Maes"
    
    # Load and preprocess the data
    df = load_and_preprocess_data(file_path)
    
    # Set the date range for which to generate HTML files
    # You can adjust these dates as needed
    start_date = datetime(2025, 4, 19)  # Today's date
    end_date = datetime(2025, 6, 30)    # Generate for next 2 months
    
    # Generate daily HTML files
    generate_daily_html_files(df, start_date, end_date, logo_path=logo_path, user_name=user_name)
    
    # Generate weekly HTML files
    generate_weekly_html_files(df, start_date, end_date, logo_path=logo_path, user_name=user_name)
    
if __name__ == "__main__":
    main()

Generated daily_html\2025-04-19.html
Generated daily_html\2025-04-20.html
Generated daily_html\2025-04-21.html
Generated daily_html\2025-04-22.html
Generated daily_html\2025-04-23.html
Generated daily_html\2025-04-24.html
Generated daily_html\2025-04-25.html
Generated daily_html\2025-04-26.html
Generated daily_html\2025-04-27.html
Generated daily_html\2025-04-28.html
Generated daily_html\2025-04-29.html
Generated daily_html\2025-04-30.html
Generated daily_html\2025-05-01.html
Generated daily_html\2025-05-02.html
Generated daily_html\2025-05-03.html
Generated daily_html\2025-05-04.html
Generated daily_html\2025-05-05.html
Generated daily_html\2025-05-06.html
Generated daily_html\2025-05-07.html
Generated daily_html\2025-05-08.html
Generated daily_html\2025-05-09.html
Generated daily_html\2025-05-10.html
Generated daily_html\2025-05-11.html
Generated daily_html\2025-05-12.html
Generated daily_html\2025-05-13.html
Generated daily_html\2025-05-14.html
Generated daily_html\2025-05-15.html
G

## Daily Sync Script for Tablet

In [6]:
# import shutil
# import os
# from datetime import datetime
# import smtplib
# from email.mime.multipart import MIMEMultipart
# from email.mime.text import MIMEText
# from email.mime.application import MIMEApplication

# def copy_today_html_to_tablet(html_dir="daily_html", tablet_path=None):
#     """Copy today's HTML file to a tablet or external device"""
#     if not tablet_path:
#         print("No tablet path specified. Please provide the path to your tablet.")
#         return
    
#     # Get today's date
#     today = datetime.now().strftime('%Y-%m-%d')
    
#     # Construct the file path
#     today_file = os.path.join(html_dir, f"{today}.html")
    
#     # Check if the file exists
#     if not os.path.exists(today_file):
#         print(f"HTML file for today ({today}) not found.")
#         return
    
#     # Check if the tablet path exists
#     if not os.path.exists(tablet_path):
#         print(f"Tablet path {tablet_path} not found.")
#         return
    
#     # Copy the file to the tablet
#     destination = os.path.join(tablet_path, "today_schedule.html")
#     try:
#         shutil.copy2(today_file, destination)
#         print(f"Successfully copied today's schedule to {destination}")
#     except Exception as e:
#         print(f"Error copying file: {e}")

# def email_today_html(html_dir="daily_html", email_to=None, smtp_server=None, 
#                     smtp_port=587, smtp_user=None, smtp_password=None):
#     """Email today's HTML file"""
#     if not email_to or not smtp_server or not smtp_user or not smtp_password:
#         print("Missing email configuration. Please provide all required parameters.")
#         return
    
#     # Get today's date
#     today = datetime.now().strftime('%Y-%m-%d')
    
#     # Construct the file path
#     today_file = os.path.join(html_dir, f"{today}.html")
    
#     # Check if the file exists
#     if not os.path.exists(today_file):
#         print(f"HTML file for today ({today}) not found.")
#         return
    
#     # Read the HTML content
#     with open(today_file, 'r', encoding='utf-8') as f:
#         html_content = f.read()
    
#     # Create the email
#     msg = MIMEMultipart()
#     msg['From'] = smtp_user
#     msg['To'] = email_to
#     msg['Subject'] = f"La Ferme de la Cour - Résumé du {datetime.now().strftime('%d/%m/%Y')}"
    
#     # Attach the HTML content
#     msg.attach(MIMEText(html_content, 'html'))
    
#     # Attach the HTML file
#     with open(today_file, 'rb') as f:
#         attachment = MIMEApplication(f.read(), _subtype="html")
#         attachment.add_header('Content-Disposition', 'attachment', filename=f"schedule_{today}.html")
#         msg.attach(attachment)
    
#     # Send the email
#     try:
#         server = smtplib.SMTP(smtp_server, smtp_port)
#         server.starttls()
#         server.login(smtp_user, smtp_password)
#         server.send_message(msg)
#         server.quit()
#         print(f"Successfully sent email to {email_to}")
#     except Exception as e:
#         print(f"Error sending email: {e}")

# def setup_tablet_display():
#     """Instructions for setting up a tablet to display the HTML files"""
#     print("""
#     To set up your tablet to display the daily HTML files:
    
#     1. Option 1 - Direct File Access:
#        - Connect your tablet to your computer regularly
#        - Use this script to copy the daily file to the tablet
#        - Create a shortcut on the tablet home screen to the HTML file
    
#     2. Option 2 - Email:
#        - Configure this script with your email settings
#        - Email the HTML file to an address accessible on the tablet
#        - Open the email each day on the tablet
    
#     3. Option 3 - Web Hosting (recommended):
#        - Upload the HTML files to a simple web server (can be local)
#        - Create a bookmark on the tablet to access the day's file
#        - Set up a daily task to update the server

#     4. Option 4 - Cloud Storage:
#        - Save files to Dropbox, Google Drive, or similar
#        - Access via the cloud storage app on the tablet
#        - Set up auto-sync on the tablet
    
#     5. Option 5 - Dedicated Apps:
#        - Use apps like 'Fully Kiosk Browser' to display web content
#        - Set up the app to auto-refresh daily
#     """)

# if __name__ == "__main__":
#     # Example usage
#     # copy_today_html_to_tablet(tablet_path="/path/to/tablet")
    
#     # Example email usage
#     # email_today_html(
#     #     email_to="owner@example.com",
#     #     smtp_server="smtp.gmail.com",
#     #     smtp_user="your-email@gmail.com",
#     #     smtp_password="your-password"
#     # )
    
#     # Show setup instructions
#     setup_tablet_display()

In [7]:
# import os
# import subprocess
# from datetime import datetime
# import time
# import schedule

# def push_to_github(html_dir="daily_html", repo_path="/path/to/your/github/repo"):
#     """Copy today's HTML file to GitHub repository and push changes"""
#     # Get today's date
#     today = datetime.now().strftime('%Y-%m-%d')
    
#     # Construct the file path
#     today_file = os.path.join(html_dir, f"{today}.html")
    
#     # Check if the file exists
#     if not os.path.exists(today_file):
#         print(f"HTML file for today ({today}) not found.")
#         return
    
#     # Copy the file to the GitHub repository
#     dest_file = os.path.join(repo_path, "index.html")  # Main page always shows today's schedule
#     history_file = os.path.join(repo_path, "history", f"{today}.html")  # Archive copy
    
#     # Ensure history directory exists
#     os.makedirs(os.path.join(repo_path, "history"), exist_ok=True)
    
#     # Copy files
#     try:
#         with open(today_file, 'r', encoding='utf-8') as src:
#             content = src.read()
            
#         with open(dest_file, 'w', encoding='utf-8') as dest:
#             dest.write(content)
            
#         with open(history_file, 'w', encoding='utf-8') as hist:
#             hist.write(content)
            
#         # Git commands to commit and push
#         os.chdir(repo_path)
#         subprocess.run(["git", "add", "index.html", f"history/{today}.html"])
#         subprocess.run(["git", "commit", "-m", f"Update schedule for {today}"])
#         subprocess.run(["git", "push"])
        
#         print(f"Successfully pushed today's schedule to GitHub Pages")
        
#     except Exception as e:
#         print(f"Error updating GitHub repository: {e}")

# def setup_github_pages_instructions():
#     """Instructions for setting up GitHub Pages"""
#     print("""
#     To set up GitHub Pages for your daily schedules:
    
#     1. Create a GitHub account if you don't have one
#     2. Create a new repository (e.g., "guesthouse-schedule")
#     3. Clone the repository to your laptop
#     4. In repository settings, enable GitHub Pages on the main branch
#     5. Your schedule will be available at: https://[your-username].github.io/[repo-name]/
#     6. Share this URL with the guesthouse owners
    
#     Note: The index.html will always show today's schedule, and previous days
#     will be available in the history folder.
#     """)

# def schedule_daily_update():
#     """Set up a scheduled task to update the GitHub Pages repository"""
#     # Schedule the task to run daily at 00:01
#     schedule.every().day.at("00:01").do(push_to_github)
    
#     print("Scheduled daily updates. Keep this script running for automatic updates.")
    
#     # Keep the script running
#     while True:
#         schedule.run_pending()
#         time.sleep(60)  # Check every minute

# if __name__ == "__main__":
#     # Show setup instructions
#     setup_github_pages_instructions()
    
#     # Example usage
#     # push_to_github(html_dir="daily_html", repo_path="/path/to/your/github/repo")
    
#     # Uncomment to start the scheduled updates
#     # schedule_daily_update()

## Google Sheets for shared use

In [8]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# import pandas as pd
# import os
# from datetime import datetime

# def setup_google_sheets_integration():
#     """Instructions for setting up Google Sheets integration"""
#     print("""
#     To set up Google Sheets integration:
    
#     1. Go to Google Cloud Console (https://console.cloud.google.com/)
#     2. Create a new project
#     3. Enable the Google Sheets API and Google Drive API
#     4. Create service account credentials and download the JSON key file
#     5. Share your Google Sheet with the email address from the service account
    
#     Save the JSON key file securely on your laptop.
#     """)

# def get_bookings_from_gsheet(credentials_file, spreadsheet_name):
#     """Get booking data from Google Sheet"""
#     # Define the scope
#     scope = ['https://spreadsheets.google.com/feeds',
#              'https://www.googleapis.com/auth/drive']
    
#     # Authenticate
#     credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_file, scope)
#     client = gspread.authorize(credentials)
    
#     # Open the spreadsheet
#     sheet = client.open(spreadsheet_name).sheet1
    
#     # Get all data
#     data = sheet.get_all_records()
    
#     # Convert to pandas DataFrame
#     df = pd.DataFrame(data)
    
#     return df

# def create_daily_html_from_gsheet(credentials_file, spreadsheet_name, output_dir="daily_html"):
#     """Create daily HTML files based on Google Sheet data"""
#     # Make sure output directory exists
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Get booking data
#     df = get_bookings_from_gsheet(credentials_file, spreadsheet_name)
    
#     # Convert date columns to datetime if they're not already
#     date_columns = ['arrival_date', 'departure_date']
#     for col in date_columns:
#         if col in df.columns:
#             df[col] = pd.to_datetime(df[col])
    
#     # Process for today
#     today = datetime.now().date()
    
#     # Find bookings active today
#     today_bookings = df[(df['arrival_date'].dt.date <= today) & 
#                          (df['departure_date'].dt.date >= today) &
#                          (df['status'] != 'cancelled')]
    
#     # Create HTML content
#     if len(today_bookings) == 0:
#         html_content = """
#         <!DOCTYPE html>
#         <html>
#         <head>
#             <meta charset="UTF-8">
#             <title>Aujourd'hui à La Ferme de la Cour</title>
#             <style>
#                 body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
#                 h1 { color: #4CAF50; }
#                 .message { font-size: 24px; text-align: center; margin-top: 100px; }
#             </style>
#         </head>
#         <body>
#             <h1>La Ferme de la Cour - {date}</h1>
#             <div class="message">
#                 <p>Aujourd'hui c'est repos! 😊</p>
#             </div>
#         </body>
#         </html>
#         """.format(date=today.strftime('%d/%m/%Y'))
#     else:
#         # Start HTML content
#         html_content = """
#         <!DOCTYPE html>
#         <html>
#         <head>
#             <meta charset="UTF-8">
#             <title>Aujourd'hui à La Ferme de la Cour</title>
#             <style>
#                 body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
#                 h1 { color: #4CAF50; }
#                 .booking { border: 1px solid #ddd; padding: 15px; margin-bottom: 20px; border-radius: 5px; }
#                 .arrival { background-color: #e6f7ff; }
#                 .departure { background-color: #fff2e6; }
#                 .both { background-color: #e6ffe6; }
#                 .regular { background-color: #f9f9f9; }
#                 .booking-title { font-weight: bold; margin-bottom: 10px; }
#                 .booking-info { margin-left: 15px; }
#                 .action-needed { color: #d9534f; font-weight: bold; }
#             </style>
#         </head>
#         <body>
#             <h1>La Ferme de la Cour - {date}</h1>
#         """.format(date=today.strftime('%d/%m/%Y'))
        
#         # Process each booking
#         for _, booking in today_bookings.iterrows():
#             arrival_date = booking['arrival_date'].date()
#             departure_date = booking['departure_date'].date()
            
#             # Determine booking type for styling
#             if arrival_date == today and departure_date == today:
#                 booking_type = "both"
#                 special_message = "<p class='action-needed'>Préparer et nettoyer la chambre pour une réservation d'un jour.</p>"
#             elif arrival_date == today:
#                 booking_type = "arrival"
#                 special_message = "<p class='action-needed'>Préparer la chambre pour l'arrivée des invités.</p>"
#             elif departure_date == today:
#                 booking_type = "departure"
#                 special_message = "<p class='action-needed'>C'est leur dernier jour chez nous. Prévoir le nettoyage de la chambre.</p>"
#             else:
#                 booking_type = "regular"
#                 special_message = ""
            
#             # Check if it's a repeat guest
#             repeat_guest_info = ""
#             if booking.get('is_repeat_guest', False):
#                 repeat_guest_info = f"""
#                 <p><strong>Client fidèle!</strong></p>
#                 <p>Notes de la visite précédente: {booking.get('previous_stay_notes', 'Aucune note disponible')}</p>
#                 """
            
#             # Add booking to HTML
#             html_content += f"""
#             <div class="booking {booking_type}">
#                 <div class="booking-title">{booking.get('room_name', 'Chambre')} - {booking.get('guest_name', 'Invité')}</div>
#                 <div class="booking-info">
#                     <p>Séjour: {arrival_date.strftime('%d/%m/%Y')} au {departure_date.strftime('%d/%m/%Y')}</p>
#                     <p>Contact: {booking.get('guest_phone', 'Non disponible')}</p>
#                     <p>Origine de la réservation: {booking.get('booking_type', 'Non spécifié')}</p>
#                     {repeat_guest_info}
#                     {special_message}
#                 </div>
#             </div>
#             """
        
#         # Close HTML content
#         html_content += """
#         </body>
#         </html>
#         """
    
#     # Save HTML file
#     file_path = os.path.join(output_dir, f"{today.strftime('%Y-%m-%d')}.html")
#     with open(file_path, 'w', encoding='utf-8') as f:
#         f.write(html_content)
    
#     print(f"Generated HTML file for {today.strftime('%Y-%m-%d')}")
#     return file_path

# def setup_automated_workflow():
#     """Set up an automated workflow to update the schedules"""
#     print("""
#     To set up a fully automated workflow:
    
#     1. Use Task Scheduler (Windows) or cron (Mac/Linux) to run this script daily
#     2. The script will:
#        - Pull the latest booking data from Google Sheets
#        - Generate the HTML file for today
#        - Push the HTML file to GitHub Pages
    
#     This ensures the owners always have access to the latest schedule.
#     """)

# if __name__ == "__main__":
#     # Show setup instructions
#     setup_google_sheets_integration()
    
#     # Example usage
#     # create_daily_html_from_gsheet(
#     #     credentials_file="path/to/your-credentials.json",
#     #     spreadsheet_name="Guesthouse Booking Tracker"
#     # )
    
#     # Show automated workflow instructions
#     setup_automated_workflow()

## Complete Automated Solution

In [9]:
# import os
# import subprocess
# import schedule
# import time
# from datetime import datetime
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# import pandas as pd

# # Configuration - Update these values
# GOOGLE_CREDENTIALS_FILE = "path/to/your-credentials.json"
# SPREADSHEET_NAME = "Guesthouse Booking Tracker"
# HTML_OUTPUT_DIR = "daily_html"
# GITHUB_REPO_PATH = "/path/to/your/github/repo"

# def get_bookings_from_gsheet():
#     """Get booking data from Google Sheet"""
#     # Define the scope
#     scope = ['https://spreadsheets.google.com/feeds',
#              'https://www.googleapis.com/auth/drive']
    
#     # Authenticate
#     credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_CREDENTIALS_FILE, scope)
#     client = gspread.authorize(credentials)
    
#     # Open the spreadsheet
#     sheet = client.open(SPREADSHEET_NAME).sheet1
    
#     # Get all data
#     data = sheet.get_all_records()
    
#     # Convert to pandas DataFrame
#     df = pd.DataFrame(data)
    
#     return df

# def create_daily_html():
#     """Create daily HTML file based on Google Sheet data"""
#     # Make sure output directory exists
#     os.makedirs(HTML_OUTPUT_DIR, exist_ok=True)
    
#     # Get booking data
#     df = get_bookings_from_gsheet()
    
#     # Convert date columns to datetime if they're not already
#     date_columns = ['arrival_date', 'departure_date']
#     for col in date_columns:
#         if col in df.columns:
#             df[col] = pd.to_datetime(df[col])
    
#     # Process for today
#     today = datetime.now().date()
    
#     # Find bookings active today
#     today_bookings = df[(df['arrival_date'].dt.date <= today) & 
#                          (df['departure_date'].dt.date >= today) &
#                          (df['status'] != 'cancelled')]
    
#     # Create HTML content
#     if len(today_bookings) == 0:
#         html_content = """
#         <!DOCTYPE html>
#         <html>
#         <head>
#             <meta charset="UTF-8">
#             <title>Aujourd'hui à La Ferme de la Cour</title>
#             <style>
#                 body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
#                 h1 { color: #4CAF50; }
#                 .message { font-size: 24px; text-align: center; margin-top: 100px; }
#             </style>
#         </head>
#         <body>
#             <h1>La Ferme de la Cour - {date}</h1>
#             <div class="message">
#                 <p>Aujourd'hui c'est repos! 😊</p>
#             </div>
#         </body>
#         </html>
#         """.format(date=today.strftime('%d/%m/%Y'))
#     else:
#         # Start HTML content
#         html_content = """
#         <!DOCTYPE html>
#         <html>
#         <head>
#             <meta charset="UTF-8">
#             <title>Aujourd'hui à La Ferme de la Cour</title>
#             <style>
#                 body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
#                 h1 { color: #4CAF50; }
#                 .booking { border: 1px solid #ddd; padding: 15px; margin-bottom: 20px; border-radius: 5px; }
#                 .arrival { background-color: #e6f7ff; }
#                 .departure { background-color: #fff2e6; }
#                 .both { background-color: #e6ffe6; }
#                 .regular { background-color: #f9f9f9; }
#                 .booking-title { font-weight: bold; margin-bottom: 10px; }
#                 .booking-info { margin-left: 15px; }
#                 .action-needed { color: #d9534f; font-weight: bold; }
#             </style>
#         </head>
#         <body>
#             <h1>La Ferme de la Cour - {date}</h1>
#         """.format(date=today.strftime('%d/%m/%Y'))
        
#         # Process each booking
#         for _, booking in today_bookings.iterrows():
#             arrival_date = booking['arrival_date'].date()
#             departure_date = booking['departure_date'].date()
            
#             # Determine booking type for styling
#             if arrival_date == today and departure_date == today:
#                 booking_type = "both"
#                 special_message = "<p class='action-needed'>Préparer et nettoyer la chambre pour une réservation d'un jour.</p>"
#             elif arrival_date == today:
#                 booking_type = "arrival"
#                 special_message = "<p class='action-needed'>Préparer la chambre pour l'arrivée des invités.</p>"
#             elif departure_date == today:
#                 booking_type = "departure"
#                 special_message = "<p class='action-needed'>C'est leur dernier jour chez nous. Prévoir le nettoyage de la chambre.</p>"
#             else:
#                 booking_type = "regular"
#                 special_message = ""
            
#             # Check if it's a repeat guest
#             repeat_guest_info = ""
#             if booking.get('is_repeat_guest', False):
#                 repeat_guest_info = f"""
#                 <p><strong>Client fidèle!</strong></p>
#                 <p>Notes de la visite précédente: {booking.get('previous_stay_notes', 'Aucune note disponible')}</p>
#                 """
            
#             # Add booking to HTML
#             html_content += f"""
#             <div class="booking {booking_type}">
#                 <div class="booking-title">{booking.get('room_name', 'Chambre')} - {booking.get('guest_name', 'Invité')}</div>
#                 <div class="booking-info">
#                     <p>Séjour: {arrival_date.strftime('%d/%m/%Y')} au {departure_date.strftime('%d/%m/%Y')}</p>
#                     <p>Contact: {booking.get('guest_phone', 'Non disponible')}</p>
#                     <p>Origine de la réservation: {booking.get('booking_type', 'Non spécifié')}</p>
#                     {repeat_guest_info}
#                     {special_message}
#                 </div>
#             </div>
#             """
        
#         # Close HTML content
#         html_content += """
#         </body>
#         </html>
#         """
    
#     # Save HTML file
#     file_path = os.path.join(HTML_OUTPUT_DIR, f"{today.strftime('%Y-%m-%d')}.html")
#     with open(file_path, 'w', encoding='utf-8') as f:
#         f.write(html_content)
    
#     print(f"Generated HTML file for {today.strftime('%Y-%m-%d')}")
#     return file_path

# def push_to_github(html_file):
#     """Push the HTML file to GitHub Pages"""
#     # Get today's date
#     today = datetime.now().strftime('%Y-%m-%d')
    
#     # Copy the file to the GitHub repository
#     dest_file = os.path.join(GITHUB_REPO_PATH, "index.html")  # Main page always shows today's schedule
#     history_file = os.path.join(GITHUB_REPO_PATH, "history", f"{today}.html")  # Archive copy
    
#     # Ensure history directory exists
#     os.makedirs(os.path.join(GITHUB_REPO_PATH, "history"), exist_ok=True)
    
#     # Copy files
#     try:
#         with open(html_file, 'r', encoding='utf-8') as src:
#             content = src.read()
            
#         with open(dest_file, 'w', encoding='utf-8') as dest:
#             dest.write(content)
            
#         with open(history_file, 'w', encoding='utf-8') as hist:
#             hist.write(content)
            
#         # Git commands to commit and push
#         os.chdir(GITHUB_REPO_PATH)
#         subprocess.run(["git", "add", "index.html", f"history/{today}.html"])
#         subprocess.run(["git", "commit", "-m", f"Update schedule for {today}"])
#         subprocess.run(["git", "push"])
        
#         print(f"Successfully pushed today's schedule to GitHub Pages")
        
#     except Exception as e:
#         print(f"Error updating GitHub repository: {e}")

# def daily_update():
#     """Perform the daily update process"""
#     try:
#         print(f"Starting daily update at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
#         html_file = create_daily_html()
#         push_to_github(html_file)
#         print(f"Daily update completed successfully")
#     except Exception as e:
#         print(f"Error in daily update process: {e}")

# def run_scheduled_updates():
#     """Run the scheduled updates continuously"""
#     # Schedule the task to run daily at 00:01
#     schedule.every().day.at("00:01").do(daily_update)
    
#     print("Scheduled daily updates. Keep this script running for automatic updates.")
#     print(f"Next update scheduled for: {schedule.next_run()}")
    
#     # Run once immediately
#     daily_update()
    
#     # Keep the script running
#     while True:
#         schedule.run_pending()
#         time.sleep(60)  # Check every minute

# def setup_instructions():
#     """Print setup instructions"""
#     print("""
#     === GUESTHOUSE MANAGEMENT AUTOMATION SETUP ===
    
#     STEP 1: GOOGLE SHEETS SETUP
#     ---------------------------
#     1. Go to Google Cloud Console (https://console.cloud.google.com/)
#     2. Create a new project
#     3. Enable the Google Sheets API and Google Drive API
#     4. Create service account credentials and download the JSON key file
#     5. Create a Google Sheet from your Excel booking tracker
#     6. Share the sheet with the email address from the service account
#     7. Share the sheet with the guesthouse owners (with edit permissions)
    
#     STEP 2: GITHUB PAGES SETUP
#     -------------------------
#     1. Create a GitHub account if you don't have one
#     2. Create a new repository (e.g., "guesthouse-schedule")
#     3. Clone the repository to your laptop
#     4. In repository settings, enable GitHub Pages on the main branch
    
#     STEP 3: CONFIGURATION
#     --------------------
#     Update the following variables in this script:
#     - GOOGLE_CREDENTIALS_FILE: path to your Google service account JSON file
#     - SPREADSHEET_NAME: name of your Google Sheet
#     - HTML_OUTPUT_DIR: directory where HTML files will be saved
#     - GITHUB_REPO_PATH: path to your local GitHub repository
    
#     STEP 4: AUTOSTART
#     ---------------
#     1. Windows: Use Task Scheduler to run this script at startup
#        - Open Task Scheduler
#        - Create a new task
#        - Set it to run at startup
#        - Action: Start a program
#        - Program/script: python
#        - Arguments: path/to/this/script.py
    
#     2. Mac/Linux: Use crontab
#        - Add this line to crontab:
#          @reboot python /path/to/this/script.py
    
#     USAGE INSTRUCTIONS FOR OWNERS
#     ----------------------------
#     1. Access today's schedule at: https://[your-username].github.io/[repo-name]/
#     2. Add or modify bookings directly in the shared Google Sheet
#     3. The schedule will update automatically every day
#     """)

# if __name__ == "__main__":
#     # Show setup instructions
#     setup_instructions()
    
#     # Uncomment to start the scheduled updates
#     # run_scheduled_updates()